# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [7]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd

spark = SparkSession.builder.appName("Data Wrangling").getOrCreate()

user_log = spark.read.json("data/sparkify_log_small.json")

user_log.describe()

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

# Question 1

Which page did user id "" (empty string) NOT visit?

In [14]:
pages = user_log.select("page").dropDuplicates().collect()

empty_user_pages = user_log.select("page").where(user_log.userId == "").dropDuplicates().collect()

empty_user_not_visited_pages = list(set(pages) - set(empty_user_pages))

print('The user empty did not visit the following pages: ')
for row in empty_user_not_visited_pages:
    print(row['page'])

The user empty did not visit the following pages: 
NextSong
Downgrade
Logout
Settings
Save Settings
Error
Submit Downgrade
Upgrade
Submit Upgrade


# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [15]:
empty_user_pages = user_log.select("page").where(user_log.userId == "").dropDuplicates().collect()

empty_user_pages

[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]

Looking above the pages that the user id empty string visited, we can conclude that **these pages does not requires the user to be registered nor logged in.**

# Question 3

How many female users do we have in the data set?

In [18]:
user_log.select(['userId', 'gender']).where(user_log.gender == 'F').dropDuplicates().describe('gender').show()

+-------+------+
|summary|gender|
+-------+------+
|  count|   462|
|   mean|  null|
| stddev|  null|
|    min|     F|
|    max|     F|
+-------+------+



# Question 4

How many songs were played from the most played artist?

In [27]:
top_artists = user_log.where(user_log.page == "NextSong").groupBy('artist').count().sort('count', ascending=False)
top_artists = top_artists.collect()

most_played_artist = top_artists[0]['artist']
print('Most played artist: ')
print(most_played_artist)

song_count = user_log.select('song').where(user_log.page == "NextSong") \
    .where(user_log.artist == most_played_artist)

song_count.describe('song').show()

Most played artist: 
Coldplay
+-------+--------------------+
|summary|                song|
+-------+--------------------+
|  count|                  83|
|   mean|                null|
| stddev|                null|
|    min|A Rush Of Blood T...|
|    max|                 Yes|
+-------+--------------------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [51]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql import Window
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import desc
from pyspark.sql.functions import col

# create a numerical flag for wether the user is in the home page
flag_homepage_visit = udf(lambda x: 1 if x == "Home" else 0, IntegerType())

# after flagging all home visits, we use that new column to create a window between home visits
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

# add a new columns called phase with that window count
user_log_valid = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userId', 'page', 'ts') \
    .withColumn("visited_home", flag_homepage_visit(col("page"))) \
    .withColumn("phase", Fsum("visited_home").over(windowval))

# with the new column, filter only the phase 1 and the page NextSong. Then group by userId and count it!
user_log_valid.where(user_log_valid.page == 'NextSong') \
    .groupBy('userId', 'phase') \
    .agg({'phase':'count'}) \
    .agg({'count(phase)':'avg'}).show()


user_log_valid count
9473
+-----------------+
|avg(count(phase))|
+-----------------+
|6.898347107438017|
+-----------------+

